In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import json

In [ ]:
rating = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Rating/enable_rating.csv')
rating.head()

,user_id,anime_id,rating
0,1,20,0
1,1,24,0
2,1,79,0
3,1,226,0
4,1,241,0


In [ ]:
user_ids = rating['user_id'].unique()
anime_ids = rating['anime_id'].unique()

In [ ]:
traindata = {}
testset = []
rating_scale = (0, 10)
offset = 1

In [ ]:
ur_dict_0 = {}
ir_dict_0 = {}
n_users = len(user_ids)
n_ratings = 0
set_items = set()
for uid in tqdm(user_ids[:]):
    uid = int(uid)
    X_tmp = rating[rating['user_id'] == uid][['anime_id', 'rating']].values.tolist()
    if len(X_tmp) == 1:
        x_train = X_tmp
        x_test = []
    else:
        x_train, x_test = train_test_split(X_tmp, test_size=0.25, random_state=42)

    n_ratings += len(x_train)
    ur_dict_0[uid] = []
    for iid, rate in x_train:
        set_items.add(iid)
        ur_dict_0[uid].append((iid, int(rate + offset)))
        if iid not in ir_dict_0.keys():
            ir_dict_0[iid] = []
        ir_dict_0[iid].append((uid, int(rate + offset)))

    for iid, rate in x_test:
        testset.append((uid, iid, int(rate + offset)))

n_items = len(set_items)

In [ ]:
raw2inner_id_users = {}
for index, uid in tqdm(enumerate(user_ids)):
    raw2inner_id_users[int(uid)] = index

raw2inner_id_items = {}
for index, iid in tqdm(enumerate(list(set_items))):
    raw2inner_id_items[iid] = index

In [ ]:
ur_dict = {}
for uid in tqdm(ur_dict_0.keys()):
    ur_dict[raw2inner_id_users[uid]] = [(raw2inner_id_items[iid], rate) for iid, rate in ur_dict_0[uid]]

ir_dict = {}
for iid in tqdm(ir_dict_0.keys()):
    ir_dict[raw2inner_id_items[iid]] = [(raw2inner_id_users[uid], rate) for uid, rate in ir_dict_0[iid]]

In [ ]:
traindata['ur_dict'] = ur_dict
traindata['ir_dict'] = ir_dict
traindata['n_users'] = n_users
traindata['n_items'] = n_items
traindata['n_ratings'] = n_ratings
traindata['offset'] = offset
traindata['rating_scale'] = rating_scale
traindata['raw2inner_id_users'] = raw2inner_id_users
traindata['raw2inner_id_items'] = raw2inner_id_items

In [ ]:
traindata.keys()

dict_keys(['ur_dict', 'ir_dict', 'n_users', 'n_items', 'n_ratings', 'offset', 'rating_scale', 'raw2inner_id_users', 'raw2inner_id_items'])

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Rating/trainset.json', 'w+') as f:
    json.dump(traindata, f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Rating/testset.json', 'w+') as f:
    json.dump({"testset": testset}, f)